Nicholas G. Keeley, ngk3pf

This code simulates an inventory queue for a grocery store. The shared resource that this class's add and remove method access is the "inventory" variable. In order to prevent race conditions between add and remove threads, this inventory is capped at 500 items and negative balances are impossible.

In [8]:
from threading import Thread
import threading
import time

class Queue:
    def __init__(self):
        self.lock = threading.Lock() # Creating lock object for acquisition.
        self.sufficientBalanceCondition = threading.Condition(self.lock)
        self.inventory = 0 # This is the shared resource that add and remove methods threads are accessing.
        
    def add(self, amount):
        self.lock.acquire() # Allows method to acquire available lock.
        try: # Try/finally in the event that there are exceptions thrown.
            while(self.inventory >= 500): # Setting a cap on inventory at 500.
                self.sufficientBalanceCondition.wait()# Temporarily release control of lock to avoid negative inventory.
            newInv = self.inventory + amount
            self.inventory = newInv
            print("Adding: %d, new inventory is %d \n" % (amount, newInv))
            self.sufficientBalanceCondition.notifyAll()
        finally: # Always executes.
            self.lock.release() # Always releases lock for acquisition.
            
    
    def remove(self, amount):
        self.lock.acquire() # Allows method to acquire available lock.
        try:
            while(self.inventory < amount):
                self.sufficientBalanceCondition.wait() # Temporarily release control of lock to avoid negative inventory.
            newInv = self.inventory - amount
            self.inventory = newInv
            print("Removing: %d, new inventory is %d \n" % (amount, newInv))
            self.sufficientBalanceCondition.notifyAll()
        finally:
            self.lock.release() # Always releases lock for acquisition.
            
def triggerAdds(queue, amount, count):
    for i in range(count):
        queue.add(amount)
        # time.sleep(1)

def triggerRemoves(queue, amount, count):
    for i in range(count):
        queue.remove(amount)
        # time.sleep(1)
 

if __name__ == '__main__':
    
    queue = Queue()
    amount = 100
    repetitions = 2
    threads = 5

    for i in range(threads):
        t1 = Thread(target=triggerAdds, args=(queue, amount, repetitions,))
        t2 = Thread(target=triggerRemoves, args=(queue, amount, repetitions,))
        t1.start()
        t2.start()

Adding: 100, new inventory is 100 

Adding: 100, new inventory is 200 

Removing: 100, new inventory is 100 

Removing: 100, new inventory is 0 

Adding: 100, new inventory is 100 

Adding: 100, new inventory is 200 

Removing: 100, new inventory is 100 

Removing: 100, new inventory is 0 

Adding: 100, new inventory is 100 

Adding: 100, new inventory is 200 

Removing: 100, new inventory is 100 

Removing: 100, new inventory is 0 

Adding: 100, new inventory is 100 

Adding: 100, new inventory is 200 

Removing: 100, new inventory is 100 

Removing: 100, new inventory is 0 

Adding: 100, new inventory is 100 

Adding: 100, new inventory is 200 

Removing: 100, new inventory is 100 

Removing: 100, new inventory is 0 

